# 1. 엑셀데이터 불러오기

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re
import os
from pyvirtualdisplay import Display
from selenium import webdriver
import time

df_input = pd.read_csv('../input_data_1~249.csv')
df_input.iloc[0]

pl_id                   1
pl_maker            apple
pl_group            apple
pl_model_code       A2221
pl_name             아이폰11
pl_model_name    iphone11
Name: 0, dtype: object

# 2. 리뷰크롤링

In [8]:
import time
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                      #스펙테이블 아이디
    "pl_model_code": "",              #모델코드
    "pl_name": "",                    #모델영문명
    
    "star": "",                       #별점
    "market": "",                     #구입처(ex 인터파크, 11번가)
    "write_id": "",                   #작성자(일부가림처리)
    "upload_day": "",                 #업로드날짜
    "title":"",                       #리뷰제목 
    "text": "",                        #리뷰내용 
    "Target":""                       #리뷰대상이 되는 상품!!(실제랑 다를 수 있으므로)
}

crawl_data = pd.DataFrame(my_dict,index=[0])
crawl_data_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

def search_by_category(df_input,crawl_data,index,option):
    url1 = 'https://search.11st.co.kr/Search.tmall?method=getCatalogPrdSearch&catalogYN=Y&kwd='
    driver.get(url1+df_input.iloc[index][4]+"#chkCtgrNo%%1002724%%중고폰%%5$$")
    driver.implicitly_wait(3)
    
    
    #페이지에 있는 링크 다 긁어오기, 일단 스크롤기능 추가안해서 5개만 가져옴 => 5개 넘어가면 검색정확도 떨어져서 고정
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')
    link_list = soup.select(".c_card_info_top > .c_prd_name_row_1 > a")
    link_list = link_list[0:3]
    for link in link_list:
        driver.get(link.get('href'))
        driver.find_element_by_xpath('//*[@id="reviewLi"]/a')
        req = driver.page_source
        soup=BeautifulSoup(req, 'html.parser')
        review_list = soup.select("#reviewObj")
        review_list = review_list[0].select("div.cfix")
        
        for review in review_list:
            
            if not review.find('span', {'class': 'selr_star'}):
                star = ""
            else:
                star = review.find('span', {'class': 'selr_star'}).text
                
            my_dict = {
                "pl_id": df_input.iloc[index][0],                      #스펙테이블 아이디
                "pl_model_code": df_input.iloc[index][3],              #모델코드
                "pl_name": df_input.iloc[index][5],                    #모델영문명

                "star": star,                                                                #별점
                "market": "11번가",                                                          #구입처(ex 인터파크, 11번가)
                "write_id": review.find('strong', {'class': 'name'}).text,                   #작성자(일부가림처리)
                "upload_day": review.find('span', {'class': 'date'}).text,                   #업로드날짜
                "title":"",                                                                  #리뷰제목 => 11번가는 없음
                "text": review.find('span', {'class': 'summ_conts'}).text,                   #리뷰내용 
                "Target":link.text,                                         #리뷰대상이 되는 상품!!(실제랑 다를 수 있으므로)
                "URL":driver.current_url
            }
            
            crawl_data = crawl_data.append(pd.DataFrame(my_dict,index=[0]))
        
        
    return crawl_data
    

# 초기화 --------------------------------------------
display = Display(visible=0, size=(800, 600))
display.start()
driver = webdriver.Chrome()
driver.implicitly_wait(3)

for i in range(len(df_input)):
    crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,""))
    print(i)
    
crawl_data.to_csv('0326_review_11st.csv',encoding='utf-8-sig')

    
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238


In [9]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', 100)
crawl_data

pandas version:  1.1.5


,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text,Target,URL
0,,,,,,,,,,,NaN
0,,,,,,,,,,,NaN
0,1,A2221,iphone11,판매자 평점 별5개 중 5개,11번가,koa****,2020-03-07,,\n사실 배송이 정말 늦었습니다만 거의 뭐 직구폰 구매하는 기분이었습니다 서울에서 ...,애플 아이폰 11 64GB 공기계(중고),https://www.11st.co.kr/catalog/catalogDetail.t...
0,1,A2221,iphone11,판매자 평점 별5개 중 5개,11번가,jos******,2020-07-16,,\n4시 전에 주문하니 담날 받았어요. \n1년도 안되서 6s 배터리 불편하기 시작...,애플 아이폰 11 64GB 공기계(중고),https://www.11st.co.kr/catalog/catalogDetail.t...
0,1,A2221,iphone11,판매자 평점 별5개 중 5개,11번가,mys***,2020-11-30,,"\n대리점 구매시 카드할인, 요금제 강요,약정 등등 불편한 내용이 생겨서 처음으로 ...",애플 아이폰 11 64GB 공기계(중고),https://www.11st.co.kr/catalog/catalogDetail.t...
0,1,A2221,iphone11,판매자 평점 별5개 중 4개,11번가,won*******,2020-12-20,,\n화이트로 주문했는데 검수시 불량이 있어서 그린으로 교환되어 배송 받았어요. 크리...,애플 아이폰 11 64GB 공기계(중고),https://www.11st.co.kr/catalog/catalogDetail.t...
0,1,A2221,iphone11,판매자 평점 별5개 중 5개,11번가,S|5**************,2020-09-05,,\n우선 배송은 태풍이랑 겹쳐서 그런지는 몰라도 좀 걸렸어요 택배 여니까 폰마블 박...,애플 아이폰 11 64GB 공기계(중고),https://www.11st.co.kr/catalog/catalogDetail.t...
0,1,A2221,iphone11,판매자 평점 별5개 중 5개,11번가,eot***,2020-06-04,,\n제가 옆면 기스는 상관없으니 화면엔 기스 1도없는걸로 달라고했는데 진짜 저부분 ...,애플 아이폰 11 64GB 공기계(중고),https://www.11st.co.kr/catalog/catalogDetail.t...
0,1,A2221,iphone11,,11번가,wit******,2020-04-04,,\n구매후 몇일뒤 전화하는데 폰이 갑자기 뜨거워지고 \n화면전체가 파랗게 변해서 껏...,애플 아이폰 11 64GB 공기계(중고),https://www.11st.co.kr/catalog/catalogDetail.t...
0,1,A2221,iphone11,판매자 평점 별5개 중 5개,11번가,pp0****,2021-02-24,,"\n아들폰이라,,,아이폰은 좀복잡하네요 먼저구경좀하려다가 포기,,,,폰은깨끗하고 이...",애플 아이폰 11 64GB 공기계(중고),https://www.11st.co.kr/catalog/catalogDetail.t...


In [6]:
crawl_data.to_csv('review_data999.csv.csv',encoding='utf-8-sig')

# 엑셀에서 pl_id 오름차순, text_len(추가후) 내림차순 정렬하고 난 후 => file.csv

In [138]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                    #모델영문명
    "text1": "",
    "text2": "",
    "text3": ""
}

filtered_reivew = pd.DataFrame(my_dict,index=[0])
filtered_reivew_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

df_output = pd.read_csv('file.csv')
pl_ids = df_output['pl_id'].unique()
models = []
for pl_id in pl_ids:
    is_name = df_output['pl_id'] == pl_id
    model = df_output[is_name]
    model = model[model.text.str.contains(
        r'(용량)|(색)|(사이즈)|(인치)|(화면)|(무게)|(센치)|(배터리)|(카메라)|(메모리)|(성능)|(출시)|(속도)'
        + r'|([0-9]+기가)|(보급형)|(방수)|(무선충전)|(지문인식)|(스크린)|(디스플레이)|(스펙)|(내장)'
        + r'|(외장)|(컬러)|(버튼)|(단자)|(현상)|(듀얼)|(화소)|(내구성)|(발열)|(업데이트)|(C타입)'
    )]    
    
    my_dict = {
    "pl_id": pl_id,                 
    }
    
    if len(model) > 0:
        my_dict["text1"] = model.iloc[0]['text']
    if len(model) > 1:
        my_dict["text2"] = model.iloc[1]['text']
    if len(model) > 2:
        my_dict["text3"] = model.iloc[2]['text']
    
    filtered_reivew = filtered_reivew.append(pd.DataFrame(my_dict,index=[0]))

In [139]:
filtered_reivew

,pl_id,text1,text2,text3
0,,,,
0,1,생각보다 하루 늦었지만 그래도 다른 곳보다 훨씬 빠르고 좋아요!!! 할인 받아서 저...,동생 사다줬는데 매우 좋아했습니다 특히 색이 너무 이쁘고 그때 마침 4만원인가 세일...,아이폰11 퍼플색을 살려고 통신사에 갔습니다. 하지만 퍼플이나 인기 좋은 제품의 색...
0,2,추석연휴 전 주라 배송이 좀 느리긴했네요 - 그래도 콜센터에서 전화와서 기분좋게 기...,한달동안 사용해 보면서 너무 너무 만족하면서 사용하고 있어요!!!! 아이폰은 흔히 ...,배송도 바로 다음날 받아볼 수 있고 아이폰11pro를 염두해두셨다면 여기서 구입하시...
0,3,백오십에 육박하는 상품을 택배로 받으려니 덜덜...상자에 심플하게 담겨옴. 에어캡이...,XS 쓰다가 넘어왔는데 너무 좋습니다 !특히 터치 후 빠른 반응 제일 만족하고 기존...,저렴하게 구매했습니다 공식인증 애플스토어에서 판매해서 구매하는데 믿음이 갔습니다 어...
0,4,친절하시구 배송도 빠르고 개통도 후개통인데 바로 됐고 가격도 방문하는거 제외 인터넷...,스그 오래걸릴줄알았는데 다행히 월요일에 구매해서 목요일에 받았어용 처음에 온할때 화...,너무 친절하시고 처리도 정말 빨라요..! 해피콜받고 바로 다음날 제품 받아볼 수 있...
...,...,...,...,...
0,245,싸게 잘 샀습니다. 다만 핸드폰 뒷면이 강화유리로 되어있어서 떨어뜨렸는데 벌써 깨어...,갤럭시 A7 2017년 제품 잘 받았습니다. 제품 색상이 없어서 아쉽긴 했지만 저렴...,NaN
0,246,NaN,NaN,NaN
0,247,NaN,NaN,NaN
0,248,NaN,NaN,NaN


In [141]:
filtered_reivew.to_csv('filtered_reivew.csv',encoding='utf-8-sig')